In [1]:
import pandas as pd

In [2]:
# show all columns
pd.set_option('display.max_columns', None)

## Transactions

In [3]:
df_transactions = pd.read_csv('data/raw/bank_a/transactions.csv')

In [4]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121457 entries, 0 to 121456
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tran_id         121457 non-null  int64  
 1   orig_acct       121457 non-null  int64  
 2   bene_acct       121457 non-null  int64  
 3   tx_type         121457 non-null  object 
 4   base_amt        121457 non-null  float64
 5   tran_timestamp  121457 non-null  object 
 6   is_sar          121457 non-null  bool   
 7   alert_id        121457 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(2)
memory usage: 6.6+ MB


In [5]:
df_transactions['tran_timestamp'] = pd.to_datetime(df_transactions['tran_timestamp'])

In [6]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121457 entries, 0 to 121456
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   tran_id         121457 non-null  int64              
 1   orig_acct       121457 non-null  int64              
 2   bene_acct       121457 non-null  int64              
 3   tx_type         121457 non-null  object             
 4   base_amt        121457 non-null  float64            
 5   tran_timestamp  121457 non-null  datetime64[ns, UTC]
 6   is_sar          121457 non-null  bool               
 7   alert_id        121457 non-null  int64              
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int64(4), object(1)
memory usage: 6.6+ MB


## Alert_Transactions

In [7]:
df_alert_tran = pd.read_csv('data/raw/bank_a/alert_transactions.csv')
df_alert_tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   alert_id        65 non-null     int64  
 1   alert_type      65 non-null     object 
 2   is_sar          65 non-null     bool   
 3   tran_id         65 non-null     int64  
 4   orig_acct       65 non-null     int64  
 5   bene_acct       65 non-null     int64  
 6   tx_type         65 non-null     object 
 7   base_amt        65 non-null     float64
 8   tran_timestamp  65 non-null     object 
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 4.3+ KB


In [8]:
df_alert_tran.head()

,alert_id,alert_type,is_sar,tran_id,orig_acct,bene_acct,tx_type,base_amt,tran_timestamp
0,2,fan_in,True,23,424,38,TRANSFER,615.19,2017-01-01T00:00:00Z
1,0,fan_in,True,193,1238,9,TRANSFER,335.76,2017-01-01T00:00:00Z
2,1,fan_in,True,222,108,53,TRANSFER,411.40,2017-01-01T00:00:00Z
3,4,fan_out,True,8516,349,1187,TRANSFER,678.05,2017-02-07T00:00:00Z
4,0,fan_in,True,10914,93,9,TRANSFER,556.89,2017-02-17T00:00:00Z


In [9]:
df_alert_tran['tran_timestamp'] = pd.to_datetime(df_alert_tran['tran_timestamp'])
df_alert_tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   alert_id        65 non-null     int64              
 1   alert_type      65 non-null     object             
 2   is_sar          65 non-null     bool               
 3   tran_id         65 non-null     int64              
 4   orig_acct       65 non-null     int64              
 5   bene_acct       65 non-null     int64              
 6   tx_type         65 non-null     object             
 7   base_amt        65 non-null     float64            
 8   tran_timestamp  65 non-null     datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int64(4), object(2)
memory usage: 4.3+ KB


## Accounts

In [10]:
df_accs = pd.read_csv('data/raw/bank_a/accounts.csv')
df_accs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1446 entries, 0 to 1445
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acct_id           1446 non-null   int64  
 1   dsply_nm          1446 non-null   object 
 2   type              1446 non-null   object 
 3   acct_stat         1446 non-null   object 
 4   acct_rptng_crncy  1446 non-null   object 
 5   prior_sar_count   1446 non-null   bool   
 6   branch_id         1446 non-null   int64  
 7   open_dt           1446 non-null   int64  
 8   close_dt          1446 non-null   int64  
 9   initial_deposit   1446 non-null   float64
 10  tx_behavior_id    0 non-null      float64
 11  bank_id           1446 non-null   object 
 12  first_name        1446 non-null   object 
 13  last_name         1446 non-null   object 
 14  street_addr       1446 non-null   object 
 15  city              1446 non-null   object 
 16  state             1446 non-null   object 


In [11]:
df_accs.head()

,acct_id,dsply_nm,type,acct_stat,acct_rptng_crncy,prior_sar_count,branch_id,open_dt,close_dt,initial_deposit,tx_behavior_id,bank_id,first_name,last_name,street_addr,city,state,country,zip,gender,birth_date,ssn,lon,lat
0,0,C_0,I,A,USD,False,1,0,1000000,96646.86,NaN,bank,Stephanie,Collins,48764 Howard Forge Apt. 421,Vanessaside,VT,US,79393,Female,2009-04-16,076-88-5410,73.479690,60.273842
1,1,C_1,I,A,USD,False,1,0,1000000,80707.36,NaN,bank,Samantha,Cook,9387 Grimes Green Apt. 801,Pagetown,RI,US,65195,Female,2009-05-08,334-91-1032,-77.421639,62.340004
2,2,C_2,I,A,USD,False,1,0,1000000,52101.10,NaN,bank,Robin,Fernandez,711 Golden Overpass,West Andreaville,OH,US,44115,Female,2018-10-05,618-71-9628,-25.565183,29.450832
3,3,C_3,I,A,USD,False,1,0,1000000,90324.40,NaN,bank,Steven,Leblanc,9342 Lori Bypass Suite 711,East Sandra,MP,US,11019,Male,1983-07-15,700-51-8595,-32.020057,50.066962
4,4,C_4,I,A,USD,False,1,0,1000000,91267.86,NaN,bank,Mark,Bruce,969 Hayes Shore,Christopherville,NM,US,77450,Male,1943-01-14,017-94-4441,-117.114006,-30.820575


In [12]:
df_accs['open_dt'].unique()

array([0])

In [13]:
df_accs['close_dt'].unique()

array([1000000])

In [14]:
df_accs.columns.tolist()

['acct_id',
 'dsply_nm',
 'type',
 'acct_stat',
 'acct_rptng_crncy',
 'prior_sar_count',
 'branch_id',
 'open_dt',
 'close_dt',
 'initial_deposit',
 'tx_behavior_id',
 'bank_id',
 'first_name',
 'last_name',
 'street_addr',
 'city',
 'state',
 'country',
 'zip',
 'gender',
 'birth_date',
 'ssn',
 'lon',
 'lat']

In [15]:

df_trans = pd.read_csv(f'data/raw/bank_a/transactions.csv')
df_accs = pd.read_csv(f'data/raw/bank_a/accounts.csv')

df_alerts = pd.read_csv(f'data/raw/bank_a/alert_transactions.csv')
df = df_trans.copy()
df['tran_timestamp'] = pd.to_datetime(df['tran_timestamp'], utc=True, errors='coerce')

df['y'] = df['tran_id'].isin(df_alerts['tran_id']).astype('int8')

# select only relevant columns
keep_cols = [
    'acct_id',
    'initial_deposit',
    'state',
    ]
df_accs = df_accs[keep_cols].copy()

df_org = df_accs.rename(columns={c: f"orig_{c}" for c in keep_cols})
df_bene = df_accs.rename(columns={c: f"bene_{c}" for c in keep_cols})

df = df.merge(df_org, how='left', left_on='orig_acct', right_on='orig_acct_id')
df = df.merge(df_bene, how='left', left_on='bene_acct', right_on='bene_acct_id')

df = df.drop(columns=['is_sar', 'alert_id'], errors='ignore')
df = df.drop(columns=['orig_acct_id', 'bene_acct_id'], errors='ignore')
df = df.drop(columns=['tx_type'], errors='ignore')
df = df.sort_values(by='tran_timestamp').reset_index(drop=True)

In [16]:
df.head()

,tran_id,orig_acct,bene_acct,base_amt,tran_timestamp,y,orig_initial_deposit,orig_state,bene_initial_deposit,bene_state
0,1,982,23,501.95,2017-01-01 00:00:00+00:00,0,79229.08,AZ,99168.24,WY
1,162,211,740,647.80,2017-01-01 00:00:00+00:00,0,73834.14,CO,84520.52,SC
2,163,863,64,804.72,2017-01-01 00:00:00+00:00,0,89755.39,PA,50082.14,KY
3,164,36,52,692.66,2017-01-01 00:00:00+00:00,0,89970.50,KS,63002.92,UT
4,165,1394,207,590.02,2017-01-01 00:00:00+00:00,0,69667.29,MS,89779.81,VA


In [17]:
df['tran_timestamp'].min(), df['tran_timestamp'].max()

(Timestamp('2017-01-01 00:00:00+0000', tz='UTC'),
 Timestamp('2018-12-21 00:00:00+0000', tz='UTC'))

In [18]:
tmp = df.copy()
tmp["month"] = tmp["tran_timestamp"].dt.to_period("M")
print(tmp.groupby("month")["y"].sum())

month
2017-01     3
2017-02     2
2017-03    19
2017-04     2
2017-05     7
2017-06     8
2017-07     4
2017-08     2
2017-09     4
2017-10     3
2017-11     1
2017-12     6
2018-01     1
2018-02     0
2018-03     0
2018-04     1
2018-05     0
2018-06     1
2018-07     1
2018-08     0
2018-09     0
2018-10     0
2018-11     0
2018-12     0
Freq: M, Name: y, dtype: int8


/tmp/ipykernel_2433295/2815097043.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  tmp["month"] = tmp["tran_timestamp"].dt.to_period("M")


In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

df = df[df['tran_timestamp'] <= pd.Timestamp("2018-01-01", tz="UTC")].reset_index(drop=True)

df = df.drop(columns=['orig_acct', 'bene_acct'], errors='ignore')

trainval_test_cutoff = pd.Timestamp("2017-11-01", tz="UTC")
train_val_cutoff = pd.Timestamp("2017-09-01", tz="UTC")

test_mask = df['tran_timestamp'] >= trainval_test_cutoff
val_mask = (df['tran_timestamp'] >= train_val_cutoff) & (df['tran_timestamp'] < trainval_test_cutoff)
train_mask = df['tran_timestamp'] < train_val_cutoff

df_test = df[test_mask].reset_index(drop=True)
df_val = df[val_mask].reset_index(drop=True)
df_train = df[train_mask].reset_index(drop=True)


cat_cols = ["orig_state", "bene_state"]
num_cols = ["base_amt", "orig_initial_deposit", "bene_initial_deposit"]

numeric_pipe = Pipeline([
    ("scaler", StandardScaler()),
    
])

categorical_pipe = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",
)


X_train = df_train[num_cols + cat_cols]
y_train = df_train['y']

X_val = df_val[num_cols + cat_cols]
y_val = df_val['y']

X_test = df_test[num_cols + cat_cols]
y_test = df_test['y']

In [20]:
df_train.tail()

,tran_id,base_amt,tran_timestamp,y,orig_initial_deposit,orig_state,bene_initial_deposit,bene_state
55536,55410,483.35,2017-08-31 00:00:00+00:00,0,87264.85,AZ,70798.81,NY
55537,55411,874.10,2017-08-31 00:00:00+00:00,0,75820.34,MD,96021.27,PR
55538,55412,303.81,2017-08-31 00:00:00+00:00,0,63029.65,VA,73519.93,HI
55539,55413,665.54,2017-08-31 00:00:00+00:00,0,71839.01,MA,56058.86,AZ
55540,55391,516.74,2017-08-31 00:00:00+00:00,0,93104.26,WY,98268.42,LA


In [21]:
df_val.tail()

,tran_id,base_amt,tran_timestamp,y,orig_initial_deposit,orig_state,bene_initial_deposit,bene_state
12448,67870,958.39,2017-10-31 00:00:00+00:00,0,86615.79,ME,97715.52,IL
12449,67871,935.79,2017-10-31 00:00:00+00:00,0,60445.79,SD,70030.16,VA
12450,67872,916.73,2017-10-31 00:00:00+00:00,0,74617.96,AZ,97111.38,NJ
12451,67873,944.93,2017-10-31 00:00:00+00:00,0,54748.24,CO,79124.72,AZ
12452,67864,596.23,2017-10-31 00:00:00+00:00,0,82813.09,MS,55712.15,MO


In [22]:
y_val.shape

(12453,)

In [23]:
y_val.sum()

np.int64(7)

In [24]:
y_val.mean()

np.float64(0.0005621135469364812)

In [25]:
y_test.mean()

np.float64(0.0006543890810507619)

In [26]:

X_train_processed = preprocess.fit_transform(X_train)
X_val_processed = preprocess.transform(X_val)